 Попробуем предсказать цены на ноутбуки, воспользововашись линейной регрессией

In [130]:
import numpy as np
import pandas as pd
import csv

In [131]:
df = pd.read_csv('laptops.csv')

In [132]:
df

,name,price,screen_size,display_resolution,cpu,proccessor_cores,set_of_cpu,cpu_frequency,gpu,ram,memory,operating_system,weight,brand
0,Lenovo ThinkPad X1 Carbon 10 21CB006QRT,240574,14.0,1920х1200,Intel Core i5,12.0,1240P,1.7,Intel Iris Xe Graphics,16.0,SSD,Windows 11 Pro,1120.000000,Lenovo
1,Lenovo ThinkPad T14 21AH00BCRT,157648,14.0,1920х1200,Intel Core i5,10.0,1235U,1.3,Intel Iris Xe Graphics,8.0,SSD,DOS / без ОС,1478.000000,Lenovo
2,ASUS ASUS Zenbook 14 90NB0W95-M00CN0,124806,14.0,2880x1800,AMD Ryzen 5,6.0,5625U,2.3,AMD Radeon Graphics,8.0,SSD,Windows 11 Home,1390.000000,ASUS
3,F+ FLTP-5R FLTP-5R5-16512-W,51300,15.6,1920x1080 FHD,AMD Ryzen 5,6.0,5600U,2.6,AMD Radeon Vega Graphics,16.0,SSD,Windows 11 Home,1760.000000,F+
4,HP ProBook 445 G9 6F1U5EA,129610,14.0,1920x1080 FHD,AMD Ryzen 7,8.0,5825U,2.0,AMD Radeon Graphics,8.0,SSD,Windows 11 Pro,1380.000000,HP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036,Dream Machines RG3050-15KZ37,89077,15.6,1920x1080 FHD,Intel Core i7,14.0,12700H,2.3,NVIDIA GeForce RTX 3050,16.0,SSD,no OS (без операционной системы),2200.000000,Dream Machines
1037,HP ProBook 450 G8 45M99ES,82597,15.6,1920x1080 FHD,Intel Core i5,4.0,1135G7,2.4,Intel Iris Xe Graphics,8.0,SSD,DOS / без ОС,1800.000000,HP
1038,"Lenovo ThinkBook 14 G4 IAP 14"" 21DH00GERU",94790,15.6,1920x1080 FHD,Intel Core i5,4.0,1135G7,2.4,Intel Iris Xe Graphics,8.0,SSD,DOS / без ОС,1705.202491,Lenovo
1039,ASUS A516EA-BQ1909W 90NB0TY1-M00U40,34180,15.6,1920x1080 FHD,Intel Pentium Gold,2.0,7505,2.4,Intel UHD Graphics,4.0,SSD,Windows 11 Home,1705.202491,ASUS


In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041 entries, 0 to 1040
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                1041 non-null   object 
 1   price               1041 non-null   int64  
 2   screen_size         1041 non-null   float64
 3   display_resolution  1041 non-null   object 
 4   cpu                 1041 non-null   object 
 5   proccessor_cores    1041 non-null   float64
 6   set_of_cpu          1041 non-null   object 
 7   cpu_frequency       1041 non-null   float64
 8   gpu                 1041 non-null   object 
 9   ram                 1041 non-null   float64
 10  memory              1041 non-null   object 
 11  operating_system    1041 non-null   object 
 12  weight              1041 non-null   float64
 13  brand               1041 non-null   object 
dtypes: float64(5), int64(1), object(8)
memory usage: 114.0+ KB


In [134]:
y = df['price']

Попробуем предугадать цены на ноутбуки при помощи следующи характеристик:

In [135]:
df_new=df[['price', 'screen_size', 'proccessor_cores', 'cpu_frequency','ram','weight']]

In [136]:
df_new

,price,screen_size,proccessor_cores,cpu_frequency,ram,weight
0,240574,14.0,12.0,1.7,16.0,1120.000000
1,157648,14.0,10.0,1.3,8.0,1478.000000
2,124806,14.0,6.0,2.3,8.0,1390.000000
3,51300,15.6,6.0,2.6,16.0,1760.000000
4,129610,14.0,8.0,2.0,8.0,1380.000000
...,...,...,...,...,...,...
1036,89077,15.6,14.0,2.3,16.0,2200.000000
1037,82597,15.6,4.0,2.4,8.0,1800.000000
1038,94790,15.6,4.0,2.4,8.0,1705.202491
1039,34180,15.6,2.0,2.4,4.0,1705.202491


In [139]:
# Линеаризуем категориальные данные "memory" в фиктивные переменные, где "1" будет отображать наличие типа памяти на ноутбуке
x_dum = pd.get_dummies(df['memory'], drop_first=False)
x_dum

,HDD,HDD+SSD (Intel Optane Memory),HDD+SSD (отдельный),SSD,SSD / SSD,eMMC
0,0,0,0,1,0,0
1,0,0,0,1,0,0
2,0,0,0,1,0,0
3,0,0,0,1,0,0
4,0,0,0,1,0,0
...,...,...,...,...,...,...
1036,0,0,0,1,0,0
1037,0,0,0,1,0,0
1038,0,0,0,1,0,0
1039,0,0,0,1,0,0


In [140]:
# Уберем зависимую переменную
df_new.drop(columns=df_new.columns[0], inplace= True )
df_new

C:\Games\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,screen_size,proccessor_cores,cpu_frequency,ram,weight
0,14.0,12.0,1.7,16.0,1120.000000
1,14.0,10.0,1.3,8.0,1478.000000
2,14.0,6.0,2.3,8.0,1390.000000
3,15.6,6.0,2.6,16.0,1760.000000
4,14.0,8.0,2.0,8.0,1380.000000
...,...,...,...,...,...
1036,15.6,14.0,2.3,16.0,2200.000000
1037,15.6,4.0,2.4,8.0,1800.000000
1038,15.6,4.0,2.4,8.0,1705.202491
1039,15.6,2.0,2.4,4.0,1705.202491


In [141]:
# Объединим обе таблицы
df_new = pd.concat([df_new, x_dum], axis=1)
df_new

,screen_size,proccessor_cores,cpu_frequency,ram,weight,HDD,HDD+SSD (Intel Optane Memory),HDD+SSD (отдельный),SSD,SSD / SSD,eMMC
0,14.0,12.0,1.7,16.0,1120.000000,0,0,0,1,0,0
1,14.0,10.0,1.3,8.0,1478.000000,0,0,0,1,0,0
2,14.0,6.0,2.3,8.0,1390.000000,0,0,0,1,0,0
3,15.6,6.0,2.6,16.0,1760.000000,0,0,0,1,0,0
4,14.0,8.0,2.0,8.0,1380.000000,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1036,15.6,14.0,2.3,16.0,2200.000000,0,0,0,1,0,0
1037,15.6,4.0,2.4,8.0,1800.000000,0,0,0,1,0,0
1038,15.6,4.0,2.4,8.0,1705.202491,0,0,0,1,0,0
1039,15.6,2.0,2.4,4.0,1705.202491,0,0,0,1,0,0


In [119]:
# Посчитаем корреляции, тк лин. регрессия основана на линейной связи между переменными 
corr_matrix = df.loc[:,:].corr()
corr_matrix

,price,screen_size,proccessor_cores,cpu_frequency,ram,weight
price,1.000000,-0.088439,0.551964,0.156093,0.660137,0.091173
screen_size,-0.088439,1.000000,0.030996,0.159131,0.081286,0.579078
proccessor_cores,0.551964,0.030996,1.000000,-0.049361,0.507566,0.157675
cpu_frequency,0.156093,0.159131,-0.049361,1.000000,0.212991,0.212441
ram,0.660137,0.081286,0.507566,0.212991,1.000000,0.164863
weight,0.091173,0.579078,0.157675,0.212441,0.164863,1.000000


Больше всего коррелирует ram и proccessor_cores с price

In [120]:
# Разделим выборку на тестовую и тренировочную
np.random.seed(2023)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_new, y, test_size=0.3)

In [142]:
# Масштабируем вещественные признаки обеих выборок
from sklearn.preprocessing import StandardScaler
normalize = StandardScaler()
x_train[['screen_size', 'proccessor_cores', 'cpu_frequency', 'ram', 'weight']] = normalize.fit_transform(x_train[['screen_size', 'proccessor_cores', 'cpu_frequency', 'ram', 'weight']])
x_train

,screen_size,proccessor_cores,cpu_frequency,ram,weight,HDD,HDD+SSD (Intel Optane Memory),HDD+SSD (отдельный),SSD,SSD / SSD,eMMC
494,0.553948,-0.113350,-1.739313,-0.626120,0.052915,0,0,0,1,0,0
68,0.553948,-0.659834,0.118404,-0.626120,-0.036567,0,0,0,1,0,0
575,0.553948,-0.659834,0.118404,0.532241,-0.036567,0,0,0,1,0,0
828,0.553948,-0.113350,0.582833,0.532241,0.361495,0,0,0,1,0,0
638,0.553948,2.072585,-0.036406,-0.626120,1.364381,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
884,0.553948,2.072585,-0.810454,0.532241,0.772935,0,0,0,1,0,0
515,-1.178236,0.979617,-1.584503,0.532241,-0.036567,0,0,0,1,0,0
695,-1.178236,1.526101,0.273214,-0.626120,0.207205,0,0,0,1,0,0
454,0.553948,-1.206318,1.047262,-0.626120,-0.178520,0,0,0,1,0,0


In [143]:
x_test[['screen_size', 'proccessor_cores', 'cpu_frequency', 'ram', 'weight']] = normalize.transform(x_test[['screen_size', 'proccessor_cores', 'cpu_frequency', 'ram', 'weight']])
x_test

,screen_size,proccessor_cores,cpu_frequency,ram,weight,HDD,HDD+SSD (Intel Optane Memory),HDD+SSD (отдельный),SSD,SSD / SSD,eMMC
183,0.553948,-0.659834,1.202072,0.532241,-0.178520,0,0,0,1,0,0
460,0.553948,-0.659834,1.821311,0.532241,1.235805,0,0,0,1,0,0
168,0.553948,-0.659834,-1.120074,0.532241,0.155775,0,0,0,1,0,0
604,-1.936067,-0.659834,0.737643,-0.626120,-0.847110,0,0,0,1,0,0
50,-1.178236,-0.113350,-1.739313,-0.626120,-0.692820,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
188,0.553948,-0.113350,-1.584503,-0.626120,-0.049945,0,0,0,1,0,0
328,0.553948,-0.659834,0.118404,-0.626120,-0.036567,0,0,0,1,0,0
148,0.553948,1.526101,1.511692,0.532241,1.235805,0,0,0,1,0,0
486,-1.178236,-0.659834,0.737643,0.532241,-0.821395,0,0,0,1,0,0


In [123]:
from sklearn.linear_model import LinearRegression 
from sklearn import metrics
from sklearn.linear_model import Ridge, Lasso

model_regression = LinearRegression()
model_regression.fit(x_train, y_train)
y_pred = model_regression.predict(x_test)

In [124]:
Train = metrics.mean_squared_error(y_train, model_regression.predict(x_train))
Test = metrics.mean_squared_error(y_test, model_regression.predict(x_test))
print('Train', Train)
print('Test', Test)

Train 2318546965.0053396
Test 1619166759.7770252


In [125]:
coef = np.abs(model_regression.coef_)
coef

array([12846.44230409, 18903.81797285,  4231.78805638, 35759.23196909,
        5708.63681533,  5280.78749102,  7352.7431783 , 24774.20605721,
        1984.75271993,  9193.10377155, 30199.38567491])

Проведем регуляризацию, чтобы исключить мультиколлинеарность

In [126]:
ridge = Ridge()
ridge.fit(x_train, y_train)
Train_Ridge = metrics.mean_squared_error(y_train, ridge.predict(x_train))
Test_Ridge = metrics.mean_squared_error(y_test, ridge.predict(x_test))
print("Train Ridge", Train_Ridge)
print("Test Ridge", Test_Ridge)

lasso = Lasso()
lasso.fit(x_train, y_train)
Train_Lasso = metrics.mean_squared_error(y_train, lasso.predict(x_train))
Test_Lasso = metrics.mean_squared_error(y_test, lasso.predict(x_test))
print("Train Lasso", Train_Lasso)
print("Test Lasso", Test_Lasso)


Train Ridge 2318868108.996698
Test Ridge 1616506493.85345
Train Lasso 2318548187.3853493
Test Lasso 1619106062.2285366


In [145]:
coef_lasso = np.abs(lasso.coef_)
coef_lasso

array([12842.19953377, 18900.68078181,  4230.41651558, 35760.43082468,
        5708.81732975,  3209.95421914,  4633.81557187, 22663.12000933,
           0.        , 11056.44041987, 32037.15713378])

In [144]:
coef_ridge = np.abs(ridge.coef_)
coef_ridge

array([12757.64969205, 18890.47220794,  4272.77865904, 35740.26009221,
        5705.11044166,  4988.7867635 ,  3789.81091796, 21414.29844891,
        2257.42608897,  7641.55211257, 24808.77010676])

### Выводы
После проведения регуляризации можно заметить, что MSE почти не изменилось, из этого можно сделать вывод, что регуляризация оказалась не очень полезной. Значения MSE достаточно велики. Такое ощущение, что модель плохо обучена. А значения coef_lasso, coef_ridge и coef показывают важность каждого признака в предсказании переменной